In [1]:
import pandas as pd
import numpy as np
import re
from string import punctuation 

In [2]:
df=pd.read_csv("../data/raw/movies_meta.csv")

In [3]:
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,1995-11-16,352194034.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0
3,False,NaN,3600000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",http://www.mgm.com/title_title.do?title_star=L...,451,tt0113627,en,Leaving Las Vegas,"Ben Sanderson, an alcoholic Hollywood screenwr...",...,1995-10-27,49800000.0,112.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,I Love You... The Way You Are.,Leaving Las Vegas,False,7.1,365.0
4,False,NaN,29500000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",NaN,63,tt0114746,en,Twelve Monkeys,"In the year 2035, convict James Cole reluctant...",...,1995-12-29,168840000.0,129.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The future is history.,Twelve Monkeys,False,7.4,2470.0


In [4]:
df.shape

(4690, 24)

In [5]:
df.title.to_csv('../models/Titles.csv',index=False, header=False)

In [6]:
df_sub=pd.read_csv("../data/raw/movies_subtitles.csv")

In [7]:
df_sub.head()

,start_time,end_time,text,imdb_id
0,58.559,61.602,"BOY: All right, everyone!\nThis... is a stick-up!",tt0114709
1,61.687,63.354,Don't anybody move!,tt0114709
2,64.398,66.482,"Now, empty that safe!",tt0114709
3,68.318,71.612,"Ooh-hoo-hoo!\nMoney, money, money! (KISSING)",tt0114709
4,71.697,74.031,"Stop it! Stop it,\nyou mean, old potato!",tt0114709


In [14]:
df['title'][199]

'Return of the Jedi'

In [91]:
imdb=df.loc[df['title'] == "Toy Story"]['imdb_id']
imdb

0    tt0114709
Name: imdb_id, dtype: object

In [92]:
df_t=df_sub.loc[df_sub['imdb_id']==imdb[0]]

In [93]:
df_t

,start_time,end_time,text,imdb_id
0,58.559,61.602,"BOY: All right, everyone!\nThis... is a stick-up!",tt0114709
1,61.687,63.354,Don't anybody move!,tt0114709
2,64.398,66.482,"Now, empty that safe!",tt0114709
3,68.318,71.612,"Ooh-hoo-hoo!\nMoney, money, money! (KISSING)",tt0114709
4,71.697,74.031,"Stop it! Stop it,\nyou mean, old potato!",tt0114709
...,...,...,...,...
680741,4722.509,4725.595,♪ Our friendship will never die,tt0114709
680742,4727.139,4731.935,♪ You're gonna see it's our destiny,tt0114709
680743,4732.019,4735.980,♪ You've got a friend in me,tt0114709
680744,4736.065,4738.650,♪ You've got a friend in me,tt0114709


In [62]:
import nltk
import nltk.data
from string import punctuation 
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords

##### Removing all songs from the subtitles data 

In [63]:
df_t.text.str.contains('♪')

461123    False
461124    False
461125    False
461126    False
461127    False
          ...  
929989    False
929990    False
929991    False
929992    False
929993    False
Name: text, Length: 2400, dtype: bool

In [94]:
df_nosong=df_t[~df_t.text.str.contains('♪')]
df_nosong

,start_time,end_time,text,imdb_id
0,58.559,61.602,"BOY: All right, everyone!\nThis... is a stick-up!",tt0114709
1,61.687,63.354,Don't anybody move!,tt0114709
2,64.398,66.482,"Now, empty that safe!",tt0114709
3,68.318,71.612,"Ooh-hoo-hoo!\nMoney, money, money! (KISSING)",tt0114709
4,71.697,74.031,"Stop it! Stop it,\nyou mean, old potato!",tt0114709
...,...,...,...,...
680715,4612.024,4614.651,"-Are you?\n-(CHUCKLES) Now, Buzz,",tt0114709
680716,4614.735,4618.988,what could Andy possibly get\nthat is worse th...,tt0114709
680717,4620.324,4622.033,"ANDY ON MONITOR:\nOh, what is it? What is it?",tt0114709
680718,4622.118,4624.077,-(PUPPY BARKING)\n-ANDY: Wow! A puppy!,tt0114709


In [42]:
def text_cleaning(text):
   
    text=re.sub("\(.*?\)","",text)

    text = re.sub(r"[^A-Za-z]", " ", str(text))
    
     #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)

    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    # Remove punctuation from text
    text = "".join([c for c in text if c not in punctuation])
    stopwords = nltk.corpus.stopwords.words('english')
    text = text.split()
    text = [w for w in text if not w in stopwords]
    text = " ".join(text)
        
    text = text.split()
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in text]
    text = " ".join(lemmatized_words)
    text=text.lower()
    
    return text 

In [95]:
df_nosong['cleaned_text'] = df_nosong['text'].apply(lambda x: text_cleaning(x))


/var/folders/th/qgn0zvb111n4qgsdq1z81tj00000gn/T/ipykernel_64490/2996799719.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nosong['cleaned_text'] = df_nosong['text'].apply(lambda x: text_cleaning(x))


In [96]:
df_nosong

,start_time,end_time,text,imdb_id,cleaned_text
0,58.559,61.602,"BOY: All right, everyone!\nThis... is a stick-up!",tt0114709,boy all right everyone this stick
1,61.687,63.354,Don't anybody move!,tt0114709,don anybody move
2,64.398,66.482,"Now, empty that safe!",tt0114709,now empty safe
3,68.318,71.612,"Ooh-hoo-hoo!\nMoney, money, money! (KISSING)",tt0114709,ooh hoo hoo money money money
4,71.697,74.031,"Stop it! Stop it,\nyou mean, old potato!",tt0114709,stop stop mean old potato
...,...,...,...,...,...
680715,4612.024,4614.651,"-Are you?\n-(CHUCKLES) Now, Buzz,",tt0114709,are now buzz
680716,4614.735,4618.988,what could Andy possibly get\nthat is worse th...,tt0114709,could andy possibly get worse
680717,4620.324,4622.033,"ANDY ON MONITOR:\nOh, what is it? What is it?",tt0114709,andy on monitor oh what
680718,4622.118,4624.077,-(PUPPY BARKING)\n-ANDY: Wow! A puppy!,tt0114709,andy wow a puppy


In [98]:
df_nosong = df_nosong[df_nosong['cleaned_text'].map(len) > 0]
df_nosong

,start_time,end_time,text,imdb_id,cleaned_text
0,58.559,61.602,"BOY: All right, everyone!\nThis... is a stick-up!",tt0114709,boy all right everyone this stick
1,61.687,63.354,Don't anybody move!,tt0114709,don anybody move
2,64.398,66.482,"Now, empty that safe!",tt0114709,now empty safe
3,68.318,71.612,"Ooh-hoo-hoo!\nMoney, money, money! (KISSING)",tt0114709,ooh hoo hoo money money money
4,71.697,74.031,"Stop it! Stop it,\nyou mean, old potato!",tt0114709,stop stop mean old potato
...,...,...,...,...,...
3867,4609.647,4611.940,SERGEANT: A large box... lt's-lt's-lt's...,tt0114709,sergeant a large box lt lt lt
3868,4612.024,4614.651,"-Are you?\n-(CHUCKLES) Now, Buzz,",tt0114709,are now buzz
3869,4614.735,4618.988,what could Andy possibly get\nthat is worse th...,tt0114709,could andy possibly get worse
3870,4620.324,4622.033,"ANDY ON MONITOR:\nOh, what is it? What is it?",tt0114709,andy on monitor oh what


In [55]:
#df_t['cleaned_text'].to_csv("../data/processed/cleaned_text.csv", index=False, header=False)

In [44]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import pickle
import itertools
import os
import csv

In [45]:
#importing libraries for models and nlp tasks
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

In [46]:
tfidf_vectorizer=pickle.load(open('../tfidfvectors/tfidf_vect_undersampling.pkl','rb'))

In [99]:
test_tfidf = tfidf_vectorizer.transform(df_nosong['cleaned_text'])

In [68]:
test_model_lr=pickle.load(open('../models/lr_neutral.pkl','rb'))

In [100]:
ytest_pred=test_model_lr.predict(test_tfidf)
ytest_pred

array([6, 6, 6, ..., 6, 6, 6])

In [101]:
df_nosong['Predicted_label']=ytest_pred

/var/folders/th/qgn0zvb111n4qgsdq1z81tj00000gn/T/ipykernel_64490/3551605409.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nosong['Predicted_label']=ytest_pred


In [102]:
df_nosong

,start_time,end_time,text,imdb_id,cleaned_text,Predicted_label
0,58.559,61.602,"BOY: All right, everyone!\nThis... is a stick-up!",tt0114709,boy all right everyone this stick,6
1,61.687,63.354,Don't anybody move!,tt0114709,don anybody move,6
2,64.398,66.482,"Now, empty that safe!",tt0114709,now empty safe,6
3,68.318,71.612,"Ooh-hoo-hoo!\nMoney, money, money! (KISSING)",tt0114709,ooh hoo hoo money money money,6
4,71.697,74.031,"Stop it! Stop it,\nyou mean, old potato!",tt0114709,stop stop mean old potato,6
...,...,...,...,...,...,...
3867,4609.647,4611.940,SERGEANT: A large box... lt's-lt's-lt's...,tt0114709,sergeant a large box lt lt lt,6
3868,4612.024,4614.651,"-Are you?\n-(CHUCKLES) Now, Buzz,",tt0114709,are now buzz,6
3869,4614.735,4618.988,what could Andy possibly get\nthat is worse th...,tt0114709,could andy possibly get worse,6
3870,4620.324,4622.033,"ANDY ON MONITOR:\nOh, what is it? What is it?",tt0114709,andy on monitor oh what,6


In [30]:
df_t['Predicted_label']

461123    1
461124    1
461125    1
461126    0
461127    1
         ..
929989    1
929990    1
929991    1
929992    3
929993    1
Name: Predicted_label, Length: 2400, dtype: int64

In [53]:
emotion = pd.read_csv('../labels_prediction/emotions_neutral.csv')


In [54]:
dic_emotions=emotion.to_dict('series')

In [55]:
dic_emotions['Emotion']

0     sadness
1         joy
2        love
3       anger
4        fear
5    surprise
6     neutral
Name: Emotion, dtype: object

In [103]:
df_nosong['Predicted_emotion'] = df_nosong['Predicted_label'].map(dic_emotions['Emotion'])


/var/folders/th/qgn0zvb111n4qgsdq1z81tj00000gn/T/ipykernel_64490/1940649588.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nosong['Predicted_emotion'] = df_nosong['Predicted_label'].map(dic_emotions['Emotion'])


In [104]:
df_nosong # Toy story

,start_time,end_time,text,imdb_id,cleaned_text,Predicted_label,Predicted_emotion
0,58.559,61.602,"BOY: All right, everyone!\nThis... is a stick-up!",tt0114709,boy all right everyone this stick,6,neutral
1,61.687,63.354,Don't anybody move!,tt0114709,don anybody move,6,neutral
2,64.398,66.482,"Now, empty that safe!",tt0114709,now empty safe,6,neutral
3,68.318,71.612,"Ooh-hoo-hoo!\nMoney, money, money! (KISSING)",tt0114709,ooh hoo hoo money money money,6,neutral
4,71.697,74.031,"Stop it! Stop it,\nyou mean, old potato!",tt0114709,stop stop mean old potato,6,neutral
...,...,...,...,...,...,...,...
3867,4609.647,4611.940,SERGEANT: A large box... lt's-lt's-lt's...,tt0114709,sergeant a large box lt lt lt,6,neutral
3868,4612.024,4614.651,"-Are you?\n-(CHUCKLES) Now, Buzz,",tt0114709,are now buzz,6,neutral
3869,4614.735,4618.988,what could Andy possibly get\nthat is worse th...,tt0114709,could andy possibly get worse,6,neutral
3870,4620.324,4622.033,"ANDY ON MONITOR:\nOh, what is it? What is it?",tt0114709,andy on monitor oh what,6,neutral


In [105]:
df_nosong.groupby(['Predicted_emotion']).count() # 98.8% of data is neutral

,start_time,end_time,text,imdb_id,cleaned_text,Predicted_label
Predicted_emotion,,,,,,
anger,3,3,3,3,3,3
fear,3,3,3,3,3,3
joy,9,9,9,9,9,9
love,3,3,3,3,3,3
neutral,3282,3282,3282,3282,3282,3282
sadness,6,6,6,6,6,6
surprise,15,15,15,15,15,15


In [73]:
df_nosong # For Return of the Jedi

,start_time,end_time,text,imdb_id,cleaned_text,Predicted_label,Predicted_emotion
461123,148.899,152.360,"Command station, this is ST-321.\nCode clearan...",tt0086190,command station st code clearance blue,6,neutral
461124,152.444,155.362,We're starting our approach.\nDeactivate the s...,tt0086190,we starting approach deactivate security shield,6,neutral
461125,155.489,160.784,The security deflector shield will be deactiva...,tt0086190,the security deflector shield deactivated conf...,6,neutral
461126,160.868,161.953,Stand by.,tt0086190,stand,6,neutral
461127,163.372,165.288,You are clear to proceed.,tt0086190,you clear proceed,6,neutral
...,...,...,...,...,...,...,...
929989,7438.347,7440.265,"When he comes back,",tt0086190,when come back,6,neutral
929990,7440.350,7443.018,I won't get in the way.,tt0086190,i get way,6,neutral
929991,7444.311,7447.689,It's not like that at all.,tt0086190,it like,6,neutral
929992,7450.777,7452.694,He's my brother.,tt0086190,he brother,6,neutral


In [75]:
df_nosong.groupby(['Predicted_emotion']).count() #95% of data is neutral

,start_time,end_time,text,imdb_id,cleaned_text,Predicted_label
Predicted_emotion,,,,,,
anger,26,26,26,26,26,26
fear,8,8,8,8,8,8
joy,28,28,28,28,28,28
love,6,6,6,6,6,6
neutral,2298,2298,2298,2298,2298,2298
sadness,26,26,26,26,26,26
surprise,8,8,8,8,8,8


In [89]:
df_nosong #Harry potter 1

,start_time,end_time,text,imdb_id,cleaned_text,Predicted_label,Predicted_emotion
2639074,81.700,87.000,"I should've known that you would\nbe here, Pro...",tt0241527,i known would professor mcgonagall,6,neutral
2639075,97.900,101.400,"Good evening, Professor Dumbledore.",tt0241527,good evening professor dumbledore,6,neutral
2639076,103.599,106.899,"Are the rumors true, Albus?",tt0241527,are rumor true albus,6,neutral
2639077,106.900,112.299,"I'm afraid so, professor.\nThe good and the bad.",tt0241527,i afraid professor the good bad,6,neutral
2639078,112.299,115.799,- And the boy?\n- Hagrid is bringing him.,tt0241527,and boy hagrid bringing,6,neutral
...,...,...,...,...,...,...,...
3259946,8976.299,8980.899,"I do. But your cousin don't, do he?",tt0241527,i but cousin,6,neutral
3259947,8986.100,8989.498,"Feels strange to be going home,\ndoesn't it?",tt0241527,feels strange going home,5,surprise
3259948,8989.500,8991.699,I'm not going home.,tt0241527,i going home,6,neutral
3259949,8991.700,8994.000,Not really.,tt0241527,not really,6,neutral


In [90]:
df_nosong.groupby(['Predicted_emotion']).count() #96% of the data for neutral

,start_time,end_time,text,imdb_id,cleaned_text,Predicted_label
Predicted_emotion,,,,,,
anger,12,12,12,12,12,12
fear,27,27,27,27,27,27
joy,24,24,24,24,24,24
love,15,15,15,15,15,15
neutral,3786,3786,3786,3786,3786,3786
sadness,15,15,15,15,15,15
surprise,36,36,36,36,36,36
